In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
timetable = pd.read_csv('timetable.csv')
timetable.drop("Unnamed: 0", axis = 1, inplace = True)
timetable.MODULE = timetable.MODULE.str.strip()
timetable.NAME = timetable.NAME.str.strip()

In [3]:
timetable.head()

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK,MODULE,NAME
0,LEC,LEC/STUDIO,1,TUE,1830-2130,LT26,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
1,LEC,LEC/STUDIO,1,TUE,1830-2130,LT27,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
2,LEC,LEC/STUDIO,1,TUE,1830-2130,LKC-LT,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
3,00822,SEM,1,WED,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS
4,00823,SEM,2,WED,1030-1230,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS


In [4]:
timetable.isna().sum()

INDEX        0
TYPE        59
GROUP        1
DAY         61
TIME        61
VENUE       65
REMARK    2635
MODULE       0
NAME         0
dtype: int64

## Split time into half an hour slots

In [5]:
def periods(start, end):
    timedel = end - start
    hour = str(timedel)[7:9]
    mins = str(timedel)[10:12] 
    try: num = int(hour) * 2 + (1 if mins == '30' else 0)
    except: num = 0
    return num

def split_periods(start, end):
    no_periods = periods(start, end) 
    timeframes = [start + pd.to_timedelta('30 min') * i for i in range(no_periods)]
    
    return timeframes

In [6]:
timetable['TIMESTART'] = pd.to_datetime(timetable.TIME.str[:4], format = '%H%M')
timetable['TIMEEND'] = pd.to_datetime(timetable.TIME.str[-4:], format = '%H%M')
timetable['PERIODS'] = timetable.apply(lambda x: periods(x.TIMESTART, x.TIMEEND), axis = 1)
timetable.head()

,INDEX,TYPE,GROUP,DAY,TIME,VENUE,REMARK,MODULE,NAME,TIMESTART,TIMEEND,PERIODS
0,LEC,LEC/STUDIO,1,TUE,1830-2130,LT26,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,1900-01-01 18:30:00,1900-01-01 21:30:00,6
1,LEC,LEC/STUDIO,1,TUE,1830-2130,LT27,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,1900-01-01 18:30:00,1900-01-01 21:30:00,6
2,LEC,LEC/STUDIO,1,TUE,1830-2130,LKC-LT,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,1900-01-01 18:30:00,1900-01-01 21:30:00,6
3,00822,SEM,1,WED,0830-1030,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,1900-01-01 08:30:00,1900-01-01 10:30:00,4
4,00823,SEM,2,WED,1030-1230,S4-CL1,Teaching Wk11,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,1900-01-01 10:30:00,1900-01-01 12:30:00,4


In [7]:
# conver to dict
timedict = timetable.to_dict('records')

In [8]:
timedict[1]

{'INDEX': 'LEC',
 'TYPE': 'LEC/STUDIO',
 'GROUP': '1',
 'DAY': 'TUE',
 'TIME': '1830-2130',
 'VENUE': 'LT27',
 'REMARK': 'Teaching Wk11',
 'MODULE': 'AB0601',
 'NAME': 'COMMUNICATION MANAGEMENT FUNDAMENTALS',
 'TIMESTART': Timestamp('1900-01-01 18:30:00'),
 'TIMEEND': Timestamp('1900-01-01 21:30:00'),
 'PERIODS': 6}

In [9]:
# breaks time into 30 mins slots
# this removes all online courses!
# deep copy is needed
# holy shit this took a long time

new_timedict = []

for row in timedict:
    start = row['TIMESTART']
    end = row['TIMEEND']
    timeframes = split_periods(start, end)
    
    for timeframe in timeframes:
        new_row = deepcopy(row)
        new_row['TIMEFRAME'] = timeframe
        new_timedict.append(new_row)

new_timedict

[{'INDEX': 'LEC',
  'TYPE': 'LEC/STUDIO',
  'GROUP': '1',
  'DAY': 'TUE',
  'TIME': '1830-2130',
  'VENUE': 'LT26',
  'REMARK': 'Teaching Wk11',
  'MODULE': 'AB0601',
  'NAME': 'COMMUNICATION MANAGEMENT FUNDAMENTALS',
  'TIMESTART': Timestamp('1900-01-01 18:30:00'),
  'TIMEEND': Timestamp('1900-01-01 21:30:00'),
  'PERIODS': 6,
  'TIMEFRAME': Timestamp('1900-01-01 18:30:00')},
 {'INDEX': 'LEC',
  'TYPE': 'LEC/STUDIO',
  'GROUP': '1',
  'DAY': 'TUE',
  'TIME': '1830-2130',
  'VENUE': 'LT26',
  'REMARK': 'Teaching Wk11',
  'MODULE': 'AB0601',
  'NAME': 'COMMUNICATION MANAGEMENT FUNDAMENTALS',
  'TIMESTART': Timestamp('1900-01-01 18:30:00'),
  'TIMEEND': Timestamp('1900-01-01 21:30:00'),
  'PERIODS': 6,
  'TIMEFRAME': Timestamp('1900-01-01 19:00:00')},
 {'INDEX': 'LEC',
  'TYPE': 'LEC/STUDIO',
  'GROUP': '1',
  'DAY': 'TUE',
  'TIME': '1830-2130',
  'VENUE': 'LT26',
  'REMARK': 'Teaching Wk11',
  'MODULE': 'AB0601',
  'NAME': 'COMMUNICATION MANAGEMENT FUNDAMENTALS',
  'TIMESTART': Timesta

In [10]:
len(new_timedict)

23475

In [11]:
new_timedf = pd.DataFrame(new_timedict)
new_timedf.head()

,DAY,GROUP,INDEX,MODULE,NAME,PERIODS,REMARK,TIME,TIMEEND,TIMEFRAME,TIMESTART,TYPE,VENUE
0,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 18:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
1,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 19:00:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
2,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 19:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
3,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 20:00:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
4,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 20:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT26


In [12]:
# insert back online courses
missing = timetable[timetable.PERIODS == 0].copy()
missing['TIMEFRAME'] = np.nan
missing = missing.reindex(columns = ['DAY','GROUP','INDEX', 'MODULE','NAME','PERIODS','REMARK','TIME','TIMEEND','TIMEFRAME','TIMESTART','TYPE','VENUE'])
new_timedf = new_timedf.append(missing)
missing

,DAY,GROUP,INDEX,MODULE,NAME,PERIODS,REMARK,TIME,TIMEEND,TIMEFRAME,TIMESTART,TYPE,VENUE
200,NaN,NBS,00586,AB1402,FOUNDATIONAL EXCEL*,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
468,NaN,NBS,02001,GC0001,SUSTAINABILITY: SEEING THROUGH THE HAZE,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
469,NaN,NBS,02000,HY0001,ETHICS & MORAL REASONING,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
513,NaN,NBS,02002,ET0001,ENTERPRISE & INNOVATION,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
514,NaN,SFP,39007,ET0001,ENTERPRISE & INNOVATION,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
667,NaN,MAE,23245,ET0001,ENTERPRISE & INNOVATION,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
668,NaN,MAE,23242,GC0001,SUSTAINABILITY: SEEING THROUGH THE HAZE,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
669,NaN,COE,50020,GC0001,SUSTAINABILITY: SEEING THROUGH THE HAZE,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
698,NaN,MAE,23243,HY0001,ETHICS & MORAL REASONING,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN
805,NaN,MAE,23244,ML0001,ABSOLUTE BASICS FOR CAREER,0,Online Course,NaN,NaT,NaN,NaT,NaN,NaN


In [13]:
len(new_timedf)

23536

In [14]:
new_timedf.to_csv('new_timedf.csv')

### Manipulation

In [15]:
new_timedf = pd.read_csv('new_timedf.csv')
new_timedf.drop("Unnamed: 0", axis = 1, inplace = True)

In [16]:
new_timedf

,DAY,GROUP,INDEX,MODULE,NAME,PERIODS,REMARK,TIME,TIMEEND,TIMEFRAME,TIMESTART,TYPE,VENUE
0,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 18:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
1,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 19:00:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
2,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 19:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
3,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 20:00:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
4,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 20:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
5,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 21:00:00,1900-01-01 18:30:00,LEC/STUDIO,LT26
6,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 18:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT27
7,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 19:00:00,1900-01-01 18:30:00,LEC/STUDIO,LT27
8,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 19:30:00,1900-01-01 18:30:00,LEC/STUDIO,LT27
9,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,1900-01-01 20:00:00,1900-01-01 18:30:00,LEC/STUDIO,LT27


In [17]:
new_timedf['RESULT'] = list(zip(new_timedf.MODULE, new_timedf.NAME, new_timedf.TYPE))
# strip timeframe to only time
new_timedf.TIMEFRAME = new_timedf.TIMEFRAME.str[11:-3]
new_timedf

,DAY,GROUP,INDEX,MODULE,NAME,PERIODS,REMARK,TIME,TIMEEND,TIMEFRAME,TIMESTART,TYPE,VENUE,RESULT
0,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,18:30,1900-01-01 18:30:00,LEC/STUDIO,LT26,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
1,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,19:00,1900-01-01 18:30:00,LEC/STUDIO,LT26,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
2,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,19:30,1900-01-01 18:30:00,LEC/STUDIO,LT26,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
3,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,20:00,1900-01-01 18:30:00,LEC/STUDIO,LT26,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
4,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,20:30,1900-01-01 18:30:00,LEC/STUDIO,LT26,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
5,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,21:00,1900-01-01 18:30:00,LEC/STUDIO,LT26,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
6,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,18:30,1900-01-01 18:30:00,LEC/STUDIO,LT27,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
7,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,19:00,1900-01-01 18:30:00,LEC/STUDIO,LT27,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
8,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,19:30,1900-01-01 18:30:00,LEC/STUDIO,LT27,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."
9,TUE,1,LEC,AB0601,COMMUNICATION MANAGEMENT FUNDAMENTALS,6,Teaching Wk11,1830-2130,1900-01-01 21:30:00,20:00,1900-01-01 18:30:00,LEC/STUDIO,LT27,"(AB0601, COMMUNICATION MANAGEMENT FUNDAMENTALS..."


In [18]:
days = ['MON', 'TUE', 'WED', 'THU', 'FRI', 'SAT']
new_timedf['DAY'] = new_timedf['DAY'].astype("category", categories=days, ordered=True)

C:\Users\JIanHui\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


In [19]:
# eg for multiple slots in single venue
new_timedf[(new_timedf.MODULE == 'FE1073') & (new_timedf.DAY == 'FRI') & (new_timedf.TIME == '0930-1230')]

,DAY,GROUP,INDEX,MODULE,NAME,PERIODS,REMARK,TIME,TIMEEND,TIMEFRAME,TIMESTART,TYPE,VENUE,RESULT
15198,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk8,0930-1230,1900-01-01 12:30:00,09:30,1900-01-01 09:30:00,LAB,N3.2-B2-01,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15199,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk8,0930-1230,1900-01-01 12:30:00,10:00,1900-01-01 09:30:00,LAB,N3.2-B2-01,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15200,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk8,0930-1230,1900-01-01 12:30:00,10:30,1900-01-01 09:30:00,LAB,N3.2-B2-01,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15201,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk8,0930-1230,1900-01-01 12:30:00,11:00,1900-01-01 09:30:00,LAB,N3.2-B2-01,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15202,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk8,0930-1230,1900-01-01 12:30:00,11:30,1900-01-01 09:30:00,LAB,N3.2-B2-01,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15203,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk8,0930-1230,1900-01-01 12:30:00,12:00,1900-01-01 09:30:00,LAB,N3.2-B2-01,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15204,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk6,0930-1230,1900-01-01 12:30:00,09:30,1900-01-01 09:30:00,LAB,S2-B5c-02,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15205,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk6,0930-1230,1900-01-01 12:30:00,10:00,1900-01-01 09:30:00,LAB,S2-B5c-02,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15206,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk6,0930-1230,1900-01-01 12:30:00,10:30,1900-01-01 09:30:00,LAB,S2-B5c-02,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."
15207,FRI,AL07,32007,FE1073,AN INTRODUCTION TO ENGINEERING & PRACTICES,6,Teaching Wk6,0930-1230,1900-01-01 12:30:00,11:00,1900-01-01 09:30:00,LAB,S2-B5c-02,"(FE1073, AN INTRODUCTION TO ENGINEERING & PRAC..."


In [20]:
pivoted = new_timedf.pivot_table(index = ['DAY','TIMEFRAME'], 
                       values = 'RESULT',
                       columns = 'VENUE', 
                       aggfunc = lambda x: tuple(x))
pivoted

VENUE         2-01-TR202 3-01-TR314 3-01-TR317 3-01-TR318 3-01-TR320  \
DAY TIMEFRAME                                                          
MON 08:30            NaN        NaN        NaN        NaN        NaN   
    09:00            NaN        NaN        NaN        NaN        NaN   
    09:30            NaN        NaN        NaN        NaN        NaN   
    10:00            NaN        NaN        NaN        NaN        NaN   
    10:30            NaN        NaN        NaN        NaN        NaN   
    11:00            NaN        NaN        NaN        NaN        NaN   
    11:30            NaN        NaN        NaN        NaN        NaN   
    12:00            NaN        NaN        NaN        NaN        NaN   
    12:30            NaN        NaN        NaN        NaN        NaN   
    13:00            NaN        NaN        NaN        NaN        NaN   
    13:30            NaN        NaN        NaN        NaN        NaN   
    14:00            NaN        NaN        NaN        NaN        NaN   
    14:30            NaN        NaN        NaN        NaN        NaN   
    15:00            NaN        NaN        NaN        NaN        NaN   
    15:30            NaN        NaN        NaN        NaN        NaN   
    16:00            NaN        NaN        NaN        NaN        NaN   
    16:30            NaN        NaN        NaN        NaN        NaN   
    17:00            NaN        NaN        NaN        NaN        NaN   
    17:30            NaN        NaN        NaN        NaN        NaN   
    18:00            NaN        NaN        NaN        NaN        NaN   
    18:30            NaN        NaN        NaN        NaN        NaN   
    19:00            NaN        NaN        NaN        NaN        NaN   
    19:30            NaN        NaN        NaN        NaN        NaN   
    20:00            NaN        NaN        NaN        NaN        NaN   
    20:30            NaN        NaN        NaN        NaN        NaN   
    21:00            NaN        NaN        NaN        NaN        NaN   
    21:30            NaN        NaN        NaN        NaN        NaN   
    22:00            NaN        NaN        NaN        NaN        NaN   
TUE 08:30            NaN        NaN        NaN        NaN        NaN   
    09:00            NaN        NaN        NaN        NaN        NaN   
...                  ...        ...        ...        ...        ...   
SAT 08:00            NaN        NaN        NaN        NaN        NaN   
    08:30            NaN        NaN        NaN        NaN        NaN   
    09:00            NaN        NaN        NaN        NaN        NaN   
    09:30            NaN        NaN        NaN        NaN        NaN   
    10:00            NaN        NaN        NaN        NaN        NaN   
    10:30            NaN        NaN        NaN        NaN        NaN   
    11:00            NaN        NaN        NaN        NaN        NaN   
    11:30            NaN        NaN        NaN        NaN        NaN   
    12:00            NaN        NaN        NaN        NaN        NaN   
    12:30            NaN        NaN        NaN        NaN        NaN   
    13:00            NaN        NaN        NaN        NaN        NaN   
    13:30            NaN        NaN        NaN        NaN        NaN   
    14:00            NaN        NaN        NaN        NaN        NaN   
    14:30            NaN        NaN        NaN        NaN        NaN   
    15:00            NaN        NaN        NaN        NaN        NaN   
    15:30            NaN        NaN        NaN        NaN        NaN   
    16:00            NaN        NaN        NaN        NaN        NaN   
    16:30            NaN        NaN        NaN        NaN        NaN   
    17:00            NaN        NaN        NaN        NaN        NaN   
    17:30            NaN        NaN        NaN        NaN        NaN   
    18:00            NaN        NaN        NaN        NaN        NaN   
    18:30            NaN        NaN        NaN        NaN        NaN   
    19:00            NaN        NaN     

In [21]:
pivoted['N1.1-B5-01']

DAY  TIMEFRAME
MON  08:30                                                      NaN
     09:00                                                      NaN
     09:30        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     10:00        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     10:30        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     11:00        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     11:30        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     12:00        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     12:30                                                      NaN
     13:00                                                      NaN
     13:30        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     14:00        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     14:30        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     15:00        ((FE1073, AN INTRODUCTION TO ENGINEERING & PRA...
     15:30        ((FE1073, AN IN

In [22]:
pivoted.to_csv('pivot.csv')